### Import libraries and data

In [3]:
import pandas as pd
import re
#%pip install text2emotion
import text2emotion as te
#%pip install emoji==1.7

path = '/Users/jonzimmerman/Desktop/Data Projects/Setlists/data/lyrics/'
lyrics = pd.read_csv(path + 'lyrics_cleaned_2024-05-27.csv')
lyrics = lyrics.drop(lyrics.columns[[0]], axis=1)

lyrics.shape

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jonzimmerman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jonzimmerman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jonzimmerman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


(1150, 3)

In [4]:
lyrics.head(3)

,title,lyrics,cleaned_lyrics
0,The Less I Know the Better,162 ContributorsTranslationsFrançaisTürkçeEspa...,someone said they left together i ran out the ...
1,"New Person, Same Old Mistakes",106 ContributorsTranslationsPortuguêsItalianoF...,i can just hear them now how could you let us ...
2,Let It Happen,124 ContributorsTranslationsEspañolFrançaisDeu...,its always around me all this noise but not ne...


#### Test out text2emotion

In [77]:
text = lyrics['cleaned_lyrics'][0]
te.get_emotion(text)


{'Happy': 0.27, 'Angry': 0.05, 'Surprise': 0.32, 'Sad': 0.18, 'Fear': 0.18}

#### Test out LeXmo

In [5]:
#%pip install LeXmo
from LeXmo import LeXmo

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jonzimmerman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
text = lyrics['cleaned_lyrics'][0]
emo=LeXmo.LeXmo(text)
emo.pop('text', None)
emo

{'anger': 0.0,
 'anticipation': 0.03404255319148936,
 'disgust': 0.0,
 'fear': 0.0,
 'joy': 0.0425531914893617,
 'negative': 0.03404255319148936,
 'positive': 0.05106382978723404,
 'sadness': 0.00425531914893617,
 'surprise': 0.00851063829787234,
 'trust': 0.029787234042553193}

#### Apply text2emotion to cleaned_lyrics

In [7]:
lyrics.loc[:,'happy'] = lyrics['cleaned_lyrics'].apply(lambda cleaned_lyrics: te.get_emotion(cleaned_lyrics)['Happy'])
lyrics.loc[:,'angry'] = lyrics['cleaned_lyrics'].apply(lambda cleaned_lyrics: te.get_emotion(cleaned_lyrics)['Angry'])
lyrics.loc[:,'surprise'] = lyrics['cleaned_lyrics'].apply(lambda cleaned_lyrics: te.get_emotion(cleaned_lyrics)['Surprise'])
lyrics.loc[:,'sad'] = lyrics['cleaned_lyrics'].apply(lambda cleaned_lyrics: te.get_emotion(cleaned_lyrics)['Sad'])
lyrics.loc[:,'fear'] = lyrics['cleaned_lyrics'].apply(lambda cleaned_lyrics: te.get_emotion(cleaned_lyrics)['Fear'])

lyrics.head(3)

,title,lyrics,cleaned_lyrics,happy,angry,surprise,sad,fear
0,The Less I Know the Better,162 ContributorsTranslationsFrançaisTürkçeEspa...,someone said they left together i ran out the ...,0.22,0.04,0.33,0.19,0.22
1,"New Person, Same Old Mistakes",106 ContributorsTranslationsPortuguêsItalianoF...,i can just hear them now how could you let us ...,0.04,0.11,0.20,0.46,0.20
2,Let It Happen,124 ContributorsTranslationsEspañolFrançaisDeu...,its always around me all this noise but not ne...,0.40,0.00,0.10,0.25,0.25


#### Apply VADER to cleaned_lyrics for pos-neg spectrum

In [8]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#VADER
sid = SentimentIntensityAnalyzer()
lyrics['scores'] = lyrics['cleaned_lyrics'].apply(lambda cleaned_lyrics: sid.polarity_scores(cleaned_lyrics))
lyrics['compound']  = lyrics['scores'].apply(lambda score_dict: score_dict['compound'])
lyrics['comp_score'] = lyrics['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
lyrics.head(3)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jonzimmerman/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,title,lyrics,cleaned_lyrics,happy,angry,surprise,sad,fear,scores,compound,comp_score
0,The Less I Know the Better,162 ContributorsTranslationsFrançaisTürkçeEspa...,someone said they left together i ran out the ...,0.22,0.04,0.33,0.19,0.22,"{'neg': 0.077, 'neu': 0.73, 'pos': 0.193, 'com...",0.9804,pos
1,"New Person, Same Old Mistakes",106 ContributorsTranslationsPortuguêsItalianoF...,i can just hear them now how could you let us ...,0.04,0.11,0.20,0.46,0.20,"{'neg': 0.179, 'neu': 0.698, 'pos': 0.123, 'co...",-0.9792,neg
2,Let It Happen,124 ContributorsTranslationsEspañolFrançaisDeu...,its always around me all this noise but not ne...,0.40,0.00,0.10,0.25,0.25,"{'neg': 0.02, 'neu': 0.882, 'pos': 0.097, 'com...",0.9848,pos


#### Download data

In [10]:
#lyrics.to_csv('lyrics_cleaned_final.csv', index=False)